<a href="https://colab.research.google.com/github/Andre2503/Group8-Project4/blob/edrin/FakeNewsCNNV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense


In [2]:
from google.colab import drive
drive.mount('/content/FakeNews_Processed_Data.csv')


Mounted at /content/FakeNews_Processed_Data.csv


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Loading the Data


In [4]:
# Loading the Data
import pandas as pd


file_path = '/content/drive/My Drive/driveAndrea/Resources/FakeNews_Processed_Data.csv'
df = pd.read_csv(file_path)
print(df.head())




                                          title_text  label
0  law enforcement high alert following threats c...      1
1                         post votes hillary already      1
2  unbelievable obamas attorney general says char...      1
3  bobby jindal raised hindu uses story christian...      0
4  satan russia unvelis image terrifying new supe...      1


# Understanding the Data

In [5]:
# Look at the first few rows of the DataFrame
print(df.head())

# Get basic information about the DataFrame
print(df.info())

# Summary statistics for numeric columns
print(df.describe())


                                          title_text  label
0  law enforcement high alert following threats c...      1
1                         post votes hillary already      1
2  unbelievable obamas attorney general says char...      1
3  bobby jindal raised hindu uses story christian...      0
4  satan russia unvelis image terrifying new supe...      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title_text  72133 non-null  object
 1   label       72134 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB
None
              label
count  72134.000000
mean       0.514404
std        0.499796
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000


In [6]:
# i'll get rid of the missing value
df = df.dropna(subset=['title_text'])


# Preprocess The Data
Text cleaning
Tokenization
Sequencing
Padding

In [7]:
# text processing
texts = df['title_text'].astype(str)
labels = df['label'].values

# Initialise and fit the tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequence
max_sequence_length = 100
data = pad_sequences(sequences, maxlen=max_sequence_length)


# Spliting the Data
Divivded data into clening and testing

In [8]:
# Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


# Build the CNN Model
Construct a CNN model suitable for text classification:

In [10]:
# Build the CNN Model
import keras
from keras.layers import Dropout

model = Sequential()
model.add(Embedding(10000, 128, input_length=100))  # Adjust these parameters as needed
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))  # Another dropout layer
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Training the model
adjusted to implement early stopping

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

# Stop training when the validation loss metric has stopped improving for 2 epochs.
early_stopping = EarlyStopping(monitor='val_loss', patience=2, mode='min', restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test),  # Use test set as validation data
                    callbacks=[early_stopping])  # Add early stopping here



Epoch 1/10
1804/1804 [==============================] - 135s 74ms/step - loss: 0.3278 - accuracy: 0.8682 - val_loss: 0.1879 - val_accuracy: 0.9249
Epoch 2/10
1804/1804 [==============================] - 128s 71ms/step - loss: 0.1999 - accuracy: 0.9320 - val_loss: 0.1608 - val_accuracy: 0.9371
Epoch 3/10
1804/1804 [==============================] - 132s 73ms/step - loss: 0.1430 - accuracy: 0.9537 - val_loss: 0.1842 - val_accuracy: 0.9392
Epoch 4/10
1804/1804 [==============================] - 132s 73ms/step - loss: 0.1140 - accuracy: 0.9629 - val_loss: 0.2011 - val_accuracy: 0.9418


In [13]:
# prompt: # Evaluate the Model


acc = model.evaluate(X_test, y_test)[1]
print('Accuracy:', acc)


451/451 [==============================] - 7s 16ms/step - loss: 0.1608 - accuracy: 0.9371
Accuracy: 0.9371317625045776


In [15]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


451/451 [==============================] - 6s 13ms/step - loss: 0.3533 - accuracy: 0.9465
Test Accuracy: 0.9464892148971558


In [14]:
# Chat GPT suggested using this to test the model to predict the likelihood that new text is fake news:

new_texts = ["Is South Africa Limiting Water for White People? Proposal Gets Pushback"]
seqs = tokenizer.texts_to_sequences(new_texts)
padded_seqs = pad_sequences(seqs, maxlen=max_sequence_length)
predictions = model.predict(padded_seqs)
print(predictions)  # Closer to 1 indicates fake, closer to 0 indicates real


1/1 [==============================] - 0s 122ms/step
[[0.65148044]]


In [15]:
# Saving the model
model.save('my_CNNmodelV2.h5')

#Saving copy to Google drive
model.save('/content/drive/My Drive/path_to_my_modelV2.h5', save_format='h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
model.save('/content/drive/My Drive/driveAndrea/Resources/CNNmodelV2.keras')
#model = load_model('my_model.keras')